# Import Dependencies

In [20]:
#data tools
import pandas as pd
from pymatgen import MPRester
#from pymatgen.vis import structure_chemview as viz

#simtool loading and interface
from simtool import findInstalledSimToolNotebooks,searchForSimTool
from simtool import getSimToolInputs,getSimToolOutputs,Run

#user interface utilities 
import os, stat
import ipywidgets as widgets
from IPython.display import display
from IPython.display import clear_output

### User Create ~/.mpkey.txt if it doesn't already exist

In [1]:
#key security
try:
    user = str(input('Paste MP API key: '))
    clear_output()
    if not user.isalnum():
        raise TypeError('Wrong Key')
    if user is None:
        raise TypeError('Empty')
    with open(os.path.expanduser('~/.mpkey.txt'), 'w') as keyfile:
        keyfile.write(user)
    os.chmod(os.path.expanduser('~/.mpkey.txt'), stat.S_IREAD | stat.S_IWRITE)
    del user
    print("Success")
except:
    print("Something seems wrong with your key")

Success


## User Prompted To pick their SemiConductor of Choice
### Choice (a): query MP and fliter on properties. Dataframe is updated with each selection.

In [6]:
with open(os.path.expanduser("~/.mpkey.txt"), "r+") as file:
    apikey = file.readline()
rester = MPRester(apikey)
sc_dicts = rester.query({ "crystal_system": "cubic"},
                        ["task_id","pretty_formula","formula","elements","e_above_hull", "spacegroup.number", "band_gap", "crystal_system"])
sc_df = pd.DataFrame(sc_dicts)

In [ ]:
#if you don't, search for your structure by filtering


In [10]:
#if you know the mp-id: give it here to get a structure object
mpid = widgets.IntText(
        value=2133,
        description='MPID:',
        disabled=False
)

display(mpid)

IntText(value=2133, description='MPID:')

In [14]:
struct = rester.get_structure_by_material_id("mp-"+str(mpid.value), final = False, conventional_unit_cell=True)

In [21]:
# if you're just hunting for something we display the structure for you.
mv = viz.quick_view(struct)
mv.ball_and_stick()

RuntimeError: To use quick_view, you need to have chemview installed.

In [17]:
#This is passed to the simtool to perform simulations
POSCAR_str = struct.to(fmt = "poscar")
POSCAR_str

'Zn2 O2\n1.0\n1.592050 -2.757511 0.000000\n1.592050 2.757511 0.000000\n0.000000 0.000000 5.155100\nZn O\n2 2\ndirect\n0.666667 0.333333 0.500000 Zn\n0.333333 0.666667 0.000000 Zn\n0.666667 0.333333 0.880310 O\n0.333333 0.666667 0.380310 O\n'

### Choice (b): upload your own poscar directly. No query necessary.

In [ ]:
with open(os.Path.expanduser("~/we_know_this_dir/POSCAR")) as file:
    POSCAR_str = file.readlines()
    

# Perform Structure Relaxation and SCF and Phonon computation and spectra extraction using simtool

### Find 670raman simtool notebook and confirm

In [ ]:
simToolName = "670raman"
simToolLocation = searchForSimTool(simToolName)
for key in simToolLocation.keys():
    print(f"{key} = {simToolLocation[key]}")

In [ ]:
installedSimToolNotebooks = findInstalledSimToolNotebooks(simToolName,returnString=True)
print(installedSimToolNotebooks)

### User Set Validated Inputs
670raman will automatically activate your rest api interface to Materials Project
if you have the dotfile ".mpkey.txt" in your home directory.

Otherwise, it will attempt to generate a realistic crystal structure from your chemical discription

In [ ]:
#Enter your values with units! The simtool will make sure you know what you're talking about.
inputs = getSimToolInputs(simToolLocation)

compound = widgets.Text(
    value='ZnO',
    placeholder='chemical formula',
    description='Compound:',
    disabled=False
)
spacegroup = widgets.IntText(
    value=186,
    placeholder='space group',
    description='Space group:',
    disabled=False
) 

widgets.IntSlider(
    value=7,
    min=0,
    max=10,
    step=1,
    description='Test:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

ecutwfc = widgets.BoundedFloatText(
    value=120,
    min=50,
    max=400,
    step=10,
    description='ecutwfc:',
    disabled=False
)

ecutrho = widgets.BoundedFloatText(
    value=480,
    min=200,
    max=1600,
    step=40,
    description='ecutrho:',
    disabled=False
)

widgets.IntText(
    value=7,
    description='Any:',
    disabled=False
)

button = widgets.Button(
    description='run simtool',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='run to submit qe simtool'
)

log = widgets.Select(
    options=["DEBUG", "INFO", "WARNING", "ERROR", "CRITICAL"],
    value='DEBUG',
    # rows=10,
    description='Log Level:',
    disabled=False
)

walltime = widgets.Text(
    value='01:00:00',
    placeholder='walltime',
    description='walltime:',
    disabled=False
)

numnodes = widgets.IntText(
    value=8,
    placeholder='nodes',
    description='nodes:',
    disabled=False
) 

# These are 
pp_menu1 = widgets.Combobox(
    placeholder="choose a pseudopotential"
    options=filtered_pp_list
    description='pseudopotential 1:',
    disabled=False
) 

pp_menu2 = widgets.Combobox(
    placeholder="choose a pseudopotential"
    options=filtered_pp_list
    description='pseudopotential 2:',
    disabled=False
)

smearing = widgets.Text(
    value='fixed',
    placeholder='smearing',
    description='smearing:',
    disabled=False
) 

smearing = widgets.Select(
    options=['smearing','fixed'],
    value='fixed',
    rows = 2,
    description='smearing:',
    disabled=False
)
    
output = widgets.Output()

# display(c, s, button, output)

def runSim2l():
    inputs['loglevel'].value = log.value
    inputs['walltime'].value = walltime.value
    inputs['numnodes'].value = numnodes.value
    inputs['compound'].value = compound.value
    inputs['ecutwfc'].value = ecutwfc.value
    inputs['ecutrho'].value = ecutrho.value
    inputs['spacegroup_international'].value = spacegroup.value
    inputs['pps'].value = [pps1.value, pps2.value]
    inputs['smearing'].value = smearing.value

def on_button_clicked(b):
    with output:
        print("submitting sim2l run with formula" , compound.value, spacegroup.value)
        runSim2l()
        r = Run(simToolLocation,inputs)
        
button.on_click(on_button_clicked)

structure = widgets.VBox([compound,spacegroup])
simulation = widgets.VBox([ecutrho, ecutwfc, smearing, pps1, pps2])
run_details = widgets.VBox([walltime, numnodes, log])

accordion = widgets.VBox([widgets.Accordion(children=[structure,simulation,run_details]),button,output])
display(accordion)

In [ ]:
inputs #request documentation of inputs  if desired

In [ ]:
inputs['loglevel'].value = "DEBUG"
inputs['walltime'].value = "01:00:00"
inputs['numnodes'].value = 8
inputs['compound'].value = "ZnO"
inputs['ecutwfc'].value = 50
inputs['ecutrho'].value = 200
inputs['spacegroup_international'].value = 186
inputs['pps'].value = ['O.pbe-hgh.UPF', 'Zn.pbe-d-hgh.UPF']
inputs['smearing'].value = "fixed"

### Show User Predetermined Outputs and their Explainations

In [ ]:
outputs = getSimToolOutputs(simToolLocation)

In [ ]:
outputs

### Run simtool to obtain Predicted Raman Tensor and Spectrum Graph

In [ ]:
r = Run(simToolLocation,inputs)

In [ ]:
r.getResultSummary()

In [ ]:
print(r.read('logreport'))

In [ ]:
r.read('spectra')

In [ ]:
#check inputs
r.input_dict

In [ ]:
#find output location
print(r.outdir)